In [2]:
import PySimpleGUI as sg
from nbimporter import NotebookLoader

newAccountWindow = NotebookLoader().load_module('newAccount')
valuesToShow = []
def manageWindow():
    # Definimos el layout de la ventana
    layout = [
        [sg.Listbox(values=valuesToShow, size=(50, 10), key="list")],
        [sg.Button("New entry")]
    ]

    # Creamos la ventana
    window = sg.Window("Passwords stored", layout)

    # Creamos una lista vacía para almacenar los elementos agregados
    items = []

    # Iniciamos el loop para leer los eventos de la ventana
    while True:
        event, values = window.read()
        # Si se presiona el botón "new entry"
        if event == "New entry":
            # Abrimos una nueva ventana para ingresar el nuevo elemento
            arrayNewEntry = newAccountWindow.manageWindow()
            # Si se ingresó un valor, lo agregamos a la lista y actualizamos la lista en la ventana
            if arrayNewEntry is not None and arrayNewEntry != "":
                items.append(arrayNewEntry)
                window["list"].update(values=items)
        # Si se cierra la ventana, cerramos el loop
        if event == sg.WINDOW_CLOSED:
            break

    # Cerramos la ventana
    window.close()

#TODO FUNCIÓN QUE RECIBA VALORES Y LOS COLOQUE EN LA LISTA
if __name__ == '__main__':
    manageWindow()


Cancel {'web': '', 'user': '', 'pass': ''}
Cancel {'web': '', 'user': '', 'pass': ''}
Cancel {'web': '', 'user': '', 'pass': ''}
Cancel {'web': '', 'user': '', 'pass': ''}
Cancel {'web': '', 'user': '', 'pass': ''}
Cancel {'web': '', 'user': '', 'pass': ''}
Cancel {'web': '', 'user': '', 'pass': ''}
None None
